In [97]:
import numpy as np
import pandas as pd
import wandb
import seaborn as sns
from matplotlib import pyplot as plt

api = wandb.Api()

In [98]:
project_path = "Fairness-wiki"
runs = api.runs(project_path)

N = 9
iterations = 1000
metrics = [f"val/perplexity_{i}" for i in range(N)]
metrics.append("val/perplexity_mean")

In [99]:
run_by_trust = {}

for run in runs:
    group = run.group
    if group not in run_by_trust:
        run_by_trust[group] = []
    run_by_trust[group].append(run)

In [100]:
metrics_by_trust = {}

for trust, runs in run_by_trust.items():
    metrics_by_trust[trust] = {}
    for metric in metrics:
        metrics_by_trust[trust][metric] = np.empty(shape=(len(runs), iterations))
    for idx, run in enumerate(runs):
        data = run.history(samples=iterations, keys=metrics)
        for metric in metrics:
            metrics_by_trust[trust][metric][idx, :] = data[metric].to_numpy()

In [101]:
last_pp = {}

for trust in metrics_by_trust.keys():
    last_pp[trust] = {}
    for metric in metrics:
        last_pp[trust][metric] = np.min(np.mean(metrics_by_trust[trust][metric], axis=0))

In [112]:
df = pd.DataFrame(last_pp)
test = pd.DataFrame()
for col in df.columns:
    if col != "local":
        test[f"Gain {col}"] = df["local"] - df[col]
test[["Gain val-sim", "Gain val-sim-w-penalizer"]]

,Gain val-sim,Gain val-sim-w-penalizer
val/perplexity_0,3.302120,3.044520
val/perplexity_1,2.383217,1.038429
val/perplexity_2,2.708804,1.198379
val/perplexity_3,3.413751,2.490498
val/perplexity_4,6.127602,2.818023
val/perplexity_5,4.330587,1.956320
val/perplexity_6,13.609152,12.294129
val/perplexity_7,20.477935,14.697831
val/perplexity_8,14.918634,10.676254
val/perplexity_mean,8.668241,3.957877
